# Final Project - CMSC 35300

## Import dataset

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# might need to change this if not working
!ln -s "/content/gdrive/MyDrive/2205.dataset/output50k/csv" ./csv

In [ ]:
!wc -l csv/*.csv

      41748 csv/allergies.csv
     207359 csv/careplans.csv
    7159335 csv/claims.csv
   58875310 csv/claims_transactions.csv
    2140306 csv/conditions.csv
      29052 csv/devices.csv
    3508873 csv/encounters.csv
    6420224 csv/imaging_studies.csv
     853153 csv/immunizations.csv
    3650463 csv/medications.csv
   36215143 csv/observations.csv
       9360 csv/organizations.csv
      58840 csv/patients.csv
         11 csv/payers.csv
    2632515 csv/payer_transitions.csv
    6441465 csv/procedures.csv
      57507 csv/providers.csv
      76740 csv/supplies.csv
  128377404 total


We can see there are 58k+ patients, 2.1M+ conditions, 3.5M+ encounters, 36M+ observations, 6.4M+ imaging studies. These will be our source of features. 

## Exploratory Data Analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Patients and Conditions table

In [4]:
patients = pd.read_csv("csv/patients.csv")
conditions = pd.read_csv("csv/conditions.csv")

In [ ]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58839 entries, 0 to 58838
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   58839 non-null  object 
 1   BIRTHDATE            58839 non-null  object 
 2   DEATHDATE            8839 non-null   object 
 3   SSN                  58839 non-null  object 
 4   DRIVERS              48206 non-null  object 
 5   PASSPORT             45210 non-null  object 
 6   PREFIX               46715 non-null  object 
 7   FIRST                58839 non-null  object 
 8   LAST                 58839 non-null  object 
 9   SUFFIX               683 non-null    object 
 10  MAIDEN               15465 non-null  object 
 11  MARITAL              39782 non-null  object 
 12  RACE                 58839 non-null  object 
 13  ETHNICITY            58839 non-null  object 
 14  GENDER               58839 non-null  object 
 15  BIRTHPLACE           58839 non-null 

In [ ]:
conditions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140305 entries, 0 to 2140304
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   START        object
 1   STOP         object
 2   PATIENT      object
 3   ENCOUNTER    object
 4   CODE         int64 
 5   DESCRIPTION  object
dtypes: int64(1), object(5)
memory usage: 98.0+ MB


In [ ]:
patients.RACE.value_counts()

white       48819
black        4596
asian        3736
other         709
hawaiian      698
native        281
Name: RACE, dtype: int64

We note that majority of the ethnicity is "white", so we will just encode the race as "white" vs "non-white" for simplicity

What conditions are available in the dataset?

In [ ]:
pd.DataFrame(conditions.groupby(["CODE", "DESCRIPTION"]).nunique().sort_values("PATIENT", ascending=False)[["PATIENT"]])

,,PATIENT
CODE,DESCRIPTION,
73595000,Stress (finding),45595
160903007,Full-time employment (finding),42413
444814009,Viral sinusitis (disorder),37265
160904001,Part-time employment (finding),35081
423315002,Limited social contact (finding),34069
...,...,...
287185009,Attempted suicide - cut/stab,3
161621004,History of upper limb amputation (situation),2
47200007,Non-low risk pregnancy,2


We note some codes of interest:

- 44054006 Diabetes
- 422034002 Diabetic retinopathy associated with type II diabetes mellitus (disorder)
- 368581000119106 Neuropathy due to type 2 diabetes mellitus (disorder)
- 127013003 Diabetic renal disease (disorder)

In [5]:
DIABETES_CODE = 44054006
COMPLICATIONS_CODES = [422034002, 368581000119106, 127013003]

#### Get Diabetes Patients

In [6]:
diabetes_conditions = conditions.query("CODE == @DIABETES_CODE")# & PATIENT in @patients.Id")
diabetes_patients = diabetes_conditions.PATIENT
diabetes_conditions = diabetes_conditions.set_index("PATIENT")

In [7]:
diabetic_patients = patients.query("Id in @diabetes_patients").set_index("Id")

In [14]:
diabetic_patients.RACE.value_counts()

white       3749
black        308
asian        275
other         66
hawaiian      49
native        30
Name: RACE, dtype: int64

In [29]:
len(diabetic_patients)

4477

#### Diabetes with Complications

In [8]:
diabetes_comp_patients = conditions.query(
    "PATIENT in @diabetes_patients &CODE in @COMPLICATIONS_CODES").PATIENT.unique()

In [16]:
diabetes_comp_patients.shape

(2686,)

In [9]:
df = pd.DataFrame(index=diabetes_patients.unique())
df["DIABETES_START"] = pd.to_datetime(diabetes_conditions.loc[diabetes_patients].START, utc=True)
df["COMP1_START"] = pd.to_datetime(conditions.query("CODE == @COMPLICATIONS_CODES[0]").set_index("PATIENT").START, utc=True)
df["COMP2_START"] = pd.to_datetime(conditions.query("CODE == @COMPLICATIONS_CODES[1]").set_index("PATIENT").START, utc=True)
df["COMP3_START"] = pd.to_datetime(conditions.query("CODE == @COMPLICATIONS_CODES[2]").set_index("PATIENT").START, utc=True)
df["DEATH_DATE"] = diabetic_patients.DEATHDATE

In [10]:
df["COMP1_DIST"] = df["COMP1_START"] - df["DIABETES_START"]
df["COMP2_DIST"] = df["COMP2_START"] - df["DIABETES_START"]
df["COMP3_DIST"] = df["COMP3_START"] - df["DIABETES_START"]
df["COMP_DIST"] = df[["COMP1_DIST","COMP2_DIST","COMP3_DIST"]].min(axis=1)

In [11]:
dfcomptime = df[(df["COMP_DIST"]>np.timedelta64(5,'Y'))]

#### Observation table

In [12]:
obs = pd.read_csv("csv/observations.csv")

In [13]:
observations = obs.query("PATIENT in @diabetes_patients")
print(observations.shape)

(6258887, 9)


In [14]:
observations['DATE_ORI'] = observations['DATE']
observations['DATE'] = pd.to_datetime(observations['DATE'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
del obs

##### Exploration

We explore available fields in the observations table

In [15]:
obs_codedesc = observations.groupby(["CODE","DESCRIPTION"]).nunique() \
              .sort_values("PATIENT", ascending=False)

In [16]:
obs_codedesc

,,DATE,PATIENT,ENCOUNTER,CATEGORY,VALUE,UNITS,TYPE,DATE_ORI
CODE,DESCRIPTION,,,,,,,,
72514-3,Pain severity - 0-10 verbal numeric rating [Score] - Reported,129128,4477,127140,1,11,1,1,129128
785-6,MCH [Entitic mass] by Automated count,14376,4477,13032,1,65,1,1,14376
39156-5,Body Mass Index,87070,4477,87321,1,290,1,1,87070
9279-1,Respiratory rate,88957,4477,87843,1,281,1,1,88957
21000-5,Erythrocyte distribution width [Entitic volume] by Automated count,11247,4477,11256,1,71,1,1,11247
...,...,...,...,...,...,...,...,...,...
417181009,Estrogen+Progesterone receptor Ag [Presence] in Tissue by Immune stain,3,3,3,1,1,0,1,3
21924-6,Tumor marker Cancer,2,2,2,1,1,0,1,2
71972-4,PROMIS-10 Global Physical Health (GPH) score,2,1,2,1,2,1,1,2


In [20]:
obs_codedesc.query("ENCOUNTER > 1000")[["ENCOUNTER","TYPE"]]

,,ENCOUNTER,TYPE
CODE,DESCRIPTION,,
72514-3,Pain severity - 0-10 verbal numeric rating [Score] - Reported,127140,1
785-6,MCH [Entitic mass] by Automated count,13032,1
39156-5,Body Mass Index,87321,1
9279-1,Respiratory rate,87843,1
21000-5,Erythrocyte distribution width [Entitic volume] by Automated count,11256,1
...,...,...,...
19123-9,Magnesium [Mass/volume] in Serum or Plasma,1197,1
33762-6,NT-proBNP,1197,1
10230-1,Left ventricular Ejection fraction,1199,1


In [21]:
observations.CATEGORY.unique()

array(['vital-signs', 'laboratory', 'social-history', 'survey',
       'procedure', 'exam', nan, 'imaging', 'therapy'], dtype=object)

In [22]:
observations.TYPE.value_counts()

numeric    3800619
text       2458268
Name: TYPE, dtype: int64

There are a ton of features in the observation table. Let's just look for features with >1000 records seen. A way to prune so that we don't deal with too much missingness

In [17]:
obs_indexes = obs_codedesc.query("ENCOUNTER > 1000").index

In [24]:
obs_indexes.get_level_values(0)

Index(['72514-3', '785-6', '39156-5', '9279-1', '21000-5', '32623-1',
       '32207-3', '777-3', '786-4', '4544-3',
       ...
       '788-0', '2276-4', '2502-3', '2498-4', '2500-7', '19123-9', '33762-6',
       '10230-1', '19926-5', '74006-8'],
      dtype='object', name='CODE', length=115)

In [18]:
snomed_obs = obs_indexes.get_level_values(0)

In [19]:
snomed_obs = snomed_obs.to_list()

In [20]:
first_vals = observations.groupby(["CODE"]).first().query("CODE in @snomed_obs")

In [21]:
numeric_obs = first_vals.query("TYPE == 'numeric'")

In [22]:
numeric_obs.index

Index(['10230-1', '10834-0', '14959-1', '1742-6', '1751-7', '17861-6',
       '18262-6', '19123-9', '1920-8', '1975-2', '19926-5', '2028-9',
       '20505-4', '20565-8', '2069-3', '2075-0', '2085-9', '2093-3', '21000-5',
       '2160-0', '2276-4', '2339-0', '2345-7', '2498-4', '2500-7', '2502-3',
       '2571-8', '2708-6', '2823-3', '2885-2', '29463-7', '2947-0', '2951-2',
       '3094-0', '32207-3', '32623-1', '33762-6', '33914-3', '38208-5',
       '38483-4', '39156-5', '44261-6', '4544-3', '4548-4', '49765-1',
       '55758-7', '5792-7', '5797-6', '5803-2', '5804-0', '5811-5', '59460-6',
       '6298-4', '6299-2', '63512-8', '6690-2', '6768-6', '70274-6', '718-7',
       '72514-3', '74006-8', '75626-2', '75893-8', '76504-0', '777-3', '785-6',
       '786-4', '787-2', '788-0', '789-8', '82667-7', '8302-2', '8310-5',
       '8462-4', '8480-6', '8867-4', '89579-7', '91145-3', '91146-1',
       '9279-1'],
      dtype='object', name='CODE')

Record the SNOMED CT codes of numeric observations

In [23]:
snomed_numeric_obs = numeric_obs.index.to_list()

In [24]:
len(snomed_numeric_obs)

80

In [28]:
first_vals.query("CODE in @snomed_numeric_obs")[["DESCRIPTION"]].to_csv("csv/numeric_snomed.csv")

Categorical observations

In [24]:
snomed_categoric_obs = first_vals.query("TYPE == 'text'").index.to_list()

In [25]:
snomed_categoric_obs.remove('56799-0') # removes "Address" field
snomed_categoric_obs.remove('63586-2') # removes "Zip code"

In [ ]:
# snomed_categoric_obs.remove('69453-9') # removes cause of death

In [ ]:
chfpatients_obs.query("CODE in @snomed_categoric_obs").groupby(["CODE","VALUE"]).nunique()[["ENCOUNTER"]]

## Feature Engineering

Problem: Predict new diabetes complication in 3 years using 2 years worth of data

Outcome: diabetes complication

Features:
- observations (2 year average of vital signs, lab values)
- medications
- procedures

In [67]:
observations.DATE > dfcomptime.iloc[0].DIABETES_START

289         True
290         True
291         True
292         True
293         True
            ... 
36211995    True
36211996    True
36211997    True
36211998    True
36211999    True
Name: DATE, Length: 6258887, dtype: bool

In [36]:
data1 = pd.DataFrame()
idx = 0
for p in dfcomptime.index:
  # add "outcome=1" observation
  here = dfcomptime.loc[p]
  start_obs = here.DIABETES_START + here.COMP_DIST - np.timedelta64(5,'Y')
  end_obs = start_obs + np.timedelta64(2,'Y')
  dnew = {}
  dnew['age'] = (start_obs-pd.to_datetime(diabetic_patients.loc[p].BIRTHDATE, utc=True))
  dnew['sex_m'] = diabetic_patients.loc[p].GENDER=='M'
  dnew['race_white'] = diabetic_patients.loc[p].RACE=='white'
  dnew['outcome'] = 1
  d_obs = observations.query(
        "PATIENT == @p & CODE in @snomed_numeric_obs & DATE >= @start_obs & DATE <= @end_obs"
        )
  d_obs.sort_values("DATE")
  d_obs.VALUE = d_obs.VALUE.astype('float')
  ss = d_obs.groupby("CODE").VALUE.mean()
  dnew = pd.DataFrame({**dnew, **ss.to_dict()}, index=[idx])
  data1 = pd.concat([data1, dnew])
  idx = idx+1
#data1['age'] = data1.DIABETES_START + diabetic_patients[dfcomptime].BIRTHDATE

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [37]:
data1.to_csv("csv/28_1010_data1.csv")

In [38]:
data0 = pd.DataFrame()
idx = 0
for p in df[df.COMP_DIST >= np.timedelta64(5,'Y')].index:
  # add "outcome=0" observation
  here = df.loc[p]
  start_obs = here.DIABETES_START
  end_obs = start_obs + np.timedelta64(2,'Y')
  dnew = {}
  dnew['age'] = (start_obs-pd.to_datetime(diabetic_patients.loc[p].BIRTHDATE, utc=True))
  dnew['sex_m'] = diabetic_patients.loc[p].GENDER=='M'
  dnew['race_white'] = diabetic_patients.loc[p].RACE=='white'
  dnew['outcome'] = 1
  d_obs = observations.query(
        "PATIENT == @p & CODE in @snomed_numeric_obs & DATE >= @start_obs & DATE <= @end_obs"
        )
  d_obs.sort_values("DATE")
  d_obs.VALUE = d_obs.VALUE.astype('float')
  ss = d_obs.groupby("CODE").VALUE.mean()
  dnew = pd.DataFrame({**dnew, **ss.to_dict()}, index=[idx])
  data0 = pd.concat([data0, dnew])
  idx = idx+1
data0.to_csv("csv/28_1010_data0.csv")
#data1['age'] = data1.DIABETES_START + diabetic_patients[dfcomptime].BIRTHDATE

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [27]:
data2 = pd.DataFrame()
idx = 0
for p in df[df.COMP_DIST.isna()].index:
  # add "outcome=0" observation
  here = df.loc[p]
  start_obs = here.DIABETES_START
  end_obs = start_obs + np.timedelta64(2,'Y')
  dnew = {}
  dnew['age'] = (start_obs-pd.to_datetime(diabetic_patients.loc[p].BIRTHDATE, utc=True))
  dnew['sex_m'] = diabetic_patients.loc[p].GENDER=='M'
  dnew['race_white'] = diabetic_patients.loc[p].RACE=='white'
  dnew['outcome'] = 1
  d_obs = observations.query(
        "PATIENT == @p & CODE in @snomed_numeric_obs & DATE >= @start_obs & DATE <= @end_obs"
        )
  d_obs.sort_values("DATE")
  d_obs.VALUE = d_obs.VALUE.astype('float')
  ss = d_obs.groupby("CODE").VALUE.mean()
  dnew = pd.DataFrame({**dnew, **ss.to_dict()}, index=[idx])
  data2 = pd.concat([data2, dnew])
  idx = idx+1
data2.to_csv("csv/28_1710_data2.csv")
#data1['age'] = data1.DIABETES_START + diabetic_patients[dfcomptime].BIRTHDATE

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
